# 텍스트 정제 text cleaning
regex를 이용해서..

In [ ]:
import re
import pandas as pd

import requests
import pandas as pd
import datetime as dt

import copy

from tqdm.notebook import tqdm
import time

In [ ]:
df = pd.read_csv('savingname.csv')

In [ ]:
new_df = copy.deepcopy(df.filter(['taken_at_timestamp' ,'owner.username','edge_media_to_caption.edges',
                    'edge_media_to_parent_comment.edges', 'edge_media_preview_like.count', 'display_url' , 'shortcode'],
                   axis = 1))

In [ ]:
new_df.columns = ['timestamp', 'username', 'post_text', 'comment_text', 'likes', 'pic_url', 'shortcode']

In [ ]:
##1. 날짜정보 변환

for i in range(len(new_df)) : 
    new_df.loc[i, "timestamp"] = dt.datetime.fromtimestamp(new_df.loc[i, "timestamp"])

In [ ]:
#2. post_text정제
#지옥의 정규식

pattern1 = '\\\\n'
pattern2 = "\[|\]|\{|\}|'text'|'node'|:|['']|#"
#pattern3 = "#"
pbar = tqdm(total = len(new_df))
for i in range(len(new_df)) : 
    text = copy.deepcopy(new_df.loc[i, 'post_text'])
    
    #1차 
    a1 = re.sub(pattern1, ' ' , text)
    
    #2차
    a2 = re.sub(pattern2, ' ', a1) 
    
    #3차
#    a3 = re.sub(pattern3, ' ', a2) 
    
    new_df.loc[i, 'post_text'] = a2
    pbar.update()
    
pbar.close()

In [ ]:
new_df.head()

In [ ]:
for i in range(5) : 
    print(new_df.loc[i, 'post_text'])

In [ ]:
#3. comment_text 정제
#지옥의 정규식...


pbar = tqdm(total = len(new_df))

for i in range(len(new_df)) : 
    text = copy.deepcopy(new_df.loc[i, 'comment_text'])
    
    a = text.split(',')
    
    a2 = str()
    
    for j in range(len(a)) : 
        if bool(re.search('text', a[j])) == True : 
            a2 = a2 + a[j]
            
    a3 = re.sub("'text'|'|@[a-z0-9_.]+|:", '', a2)
    a4 = re.sub("#", ' ', a3)

    new_df.loc[i, 'comment_text'] = a4
    
    pbar.update()
    
pbar.close()

In [ ]:
for i in range(10) : 
    print(new_df.loc[i, 'comment_text'])

In [ ]:
new_df.to_csv('cleaning완료.csv', encoding = 'utf-8-sig')

In [ ]:
import os
import urllib.request as req

dir_name = "prod_imgs"
os.makedirs(dir_name, exist_ok = True) #새로 폴더 만들어 + 부드럽게 실행시킬수있는 옵션
photo_name = "name"
pbar = tqdm(total = len(new_df))
for i in range(len(new_df)) : 
    req.urlretrieve(url = new_df.loc[i,'pic_url'], 
                    filename = dir_name + '/' + photo_name + str(i) + ".jpg")
    pbar.update()
pbar.close()